In [1]:
import sys
sys.path.append('../../')

In [2]:
import random
random.seed(4444)
import pickle
import numpy as np
np.random.seed(5555)
import tensorflow as tf
tf.set_random_seed(6666)
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
from keras.optimizers import RMSprop, Adam
from utils.dataset import DataSet, LABELS
from models.textrnn import BiCuDNNGRUSeq as TextRNN
from models.sentiment_base import SCompositeModel as CompositeTextRNN
from env import *

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
WORD_SEQS_PATH = os.path.join(CACHES_PATH, 'word_seqs', 'simple')

TRAIN_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'train_seqs_padded.pkl')
VAL_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'val_seqs_padded.pkl')
TEST_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'test_seqs_padded.pkl')

SAVED_PATH = os.path.join(SAVED_MODELS_PATH, 'textrnn', 'bicudnngruseq')
if not os.path.exists(SAVED_PATH):
    os.makedirs(SAVED_PATH)

MODEL_PRE = 'bicudnngruseq_'

LR = 1e-3
EPOCHS = 100
BATCH_SIZE = 64
FACTOR = 0.2

EMBEDDING_PKL = os.path.join(WORD_SEQS_PATH, 'wem.pkl')
RESULT_CSV = os.path.join(RESULTS_PATH, 'bigru.csv')

In [4]:
raw_dataset = DataSet()
train, val, test = raw_dataset.train, raw_dataset.val, raw_dataset.test

with open(TRAIN_SEQS_PADDED_PKL, 'rb') as f:
    train_seqs_padded = pickle.load(f)
    
with open(VAL_SEQS_PADDED_PKL, 'rb') as f:
    val_seqs_padded = pickle.load(f)
    
with open(TEST_SEQS_PADDED_PKL, 'rb') as f:
    test_seqs_padded = pickle.load(f)
    
with open(EMBEDDING_PKL, 'rb') as f:
    embedding = pickle.load(f)

In [5]:
train_with_seq = pd.merge(train, train_seqs_padded, on='id')
val_with_seq = pd.merge(val, val_seqs_padded, on='id')
test_with_seq = pd.merge(test, test_seqs_padded, on='id')

In [6]:
y_cols = LABELS

seq = 'words_seq'

train_x = np.array(list(train_with_seq[seq]))
train_y = train_with_seq[y_cols]
val_x = np.array(list(val_with_seq[seq]))
val_y = val_with_seq[y_cols]
test_x = np.array(list(test_with_seq[seq]))

comps = [(TextRNN, {"max_len":train_x.shape[1], 
                    'embedding': embedding,
                    'mask_zero': False})] * len(y_cols)
# comps = [(LinXiSingleModel, {"maxlen":train_x.shape[1]})] * len(y_cols)
comp_model = CompositeTextRNN(comps)
comp_model.fit(train_x, train_y, val = (val_x, val_y), y_cols = y_cols, seq = 'word_seq', saved_path = SAVED_PATH,
               model_pre = MODEL_PRE, lr = LR, epochs = EPOCHS, batch_size = BATCH_SIZE, optimizer = RMSprop,
               factor = FACTOR)

Train on 105000 samples, validate on 15000 samples
Epoch 1/100
105000/105000 [==============================] - 212s 2ms/step - loss: 0.2700 - acc: 0.9215 - val_loss: 0.1979 - val_acc: 0.9440


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Epoch 00001: _f1_monitor improved from -inf to 0.56761, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/bicudnngruseq/bicudnngruseq_0.h5
Epoch 2/100
105000/105000 [==============================] - 212s 2ms/step - loss: 0.2063 - acc: 0.9400 - val_loss: 0.1886 - val_acc: 0.9452

Epoch 00002: _f1_monitor improved from 0.56761 to 0.58307, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/bicudnngruseq/bicudnngruseq_0.h5
Epoch 3/100
105000/105000 [==============================] - 212s 2ms/step - loss: 0.1926 - acc: 0.9428 - val_loss: 0.1984 - val_acc: 0.9402

Epoch 00003: _f1_monitor did not improve from 0.58307
Epoch 4/100
105000/105000 [==============================] - 212s 2ms/step - loss: 0.1814 - acc: 0.9458 - val_loss: 0.1954 - val_acc: 0.9433

Epoch 00004: _f1_monitor did not improve from 0.58307
Epoch 5/100
105000/105000 [==============================] - 212s 2ms/step - loss: 0.1741 - acc: 0.9479 - 

105000/105000 [==============================] - 212s 2ms/step - loss: 0.2020 - acc: 0.9352 - val_loss: 0.2830 - val_acc: 0.9141

Epoch 00013: _f1_monitor improved from 0.67263 to 0.67653, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/bicudnngruseq/bicudnngruseq_2.h5
Epoch 14/100
105000/105000 [==============================] - 212s 2ms/step - loss: 0.2020 - acc: 0.9351 - val_loss: 0.2855 - val_acc: 0.9136

Epoch 00014: _f1_monitor did not improve from 0.67653
Epoch 15/100
105000/105000 [==============================] - 212s 2ms/step - loss: 0.2012 - acc: 0.9355 - val_loss: 0.2889 - val_acc: 0.9130

Epoch 00015: _f1_monitor did not improve from 0.67653

Epoch 00015: ReduceLROnPlateau reducing learning rate to 8.000000525498762e-06.
The 2th grain: f1_score - 0.6765336033670779, acc - 0.9141333333333334
Train on 105000 samples, validate on 15000 samples
Epoch 1/100
105000/105000 [==============================] - 213s 2ms/step - loss: 0.3447 - 

105000/105000 [==============================] - 212s 2ms/step - loss: 0.0484 - acc: 0.9872 - val_loss: 0.0952 - val_acc: 0.9732

Epoch 00008: _f1_monitor did not improve from 0.72888
Epoch 9/100
105000/105000 [==============================] - 212s 2ms/step - loss: 0.0438 - acc: 0.9885 - val_loss: 0.0970 - val_acc: 0.9755

Epoch 00009: _f1_monitor did not improve from 0.72888

Epoch 00009: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
The 5th grain: f1_score - 0.7288753830611063, acc - 0.9754666666666667
Train on 105000 samples, validate on 15000 samples
Epoch 1/100
105000/105000 [==============================] - 213s 2ms/step - loss: 0.3427 - acc: 0.8977 - val_loss: 0.2665 - val_acc: 0.9165

Epoch 00001: _f1_monitor improved from -inf to 0.58533, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/bicudnngruseq/bicudnngruseq_6.h5
Epoch 2/100
105000/105000 [==============================] - 212s 2ms/step - loss: 0.2525 - acc:

105000/105000 [==============================] - 212s 2ms/step - loss: 0.2616 - acc: 0.9106 - val_loss: 0.3600 - val_acc: 0.8766

Epoch 00007: _f1_monitor improved from 0.64673 to 0.65321, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/bicudnngruseq/bicudnngruseq_8.h5
Epoch 8/100
105000/105000 [==============================] - 212s 2ms/step - loss: 0.2492 - acc: 0.9157 - val_loss: 0.3595 - val_acc: 0.8761

Epoch 00008: _f1_monitor did not improve from 0.65321
Epoch 9/100
105000/105000 [==============================] - 212s 2ms/step - loss: 0.2383 - acc: 0.9198 - val_loss: 0.3710 - val_acc: 0.8679

Epoch 00009: _f1_monitor did not improve from 0.65321

Epoch 00009: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
The 8th grain: f1_score - 0.6532072552649792, acc - 0.8766
Train on 105000 samples, validate on 15000 samples
Epoch 1/100
105000/105000 [==============================] - 213s 2ms/step - loss: 0.5407 - acc: 0.7950 -

105000/105000 [==============================] - 212s 2ms/step - loss: 0.3479 - acc: 0.8767 - val_loss: 0.4459 - val_acc: 0.8447

Epoch 00008: _f1_monitor did not improve from 0.67942
Epoch 9/100
105000/105000 [==============================] - 212s 2ms/step - loss: 0.3444 - acc: 0.8778 - val_loss: 0.4289 - val_acc: 0.8463

Epoch 00009: _f1_monitor did not improve from 0.67942

Epoch 00009: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
The 11th grain: f1_score - 0.679419608307052, acc - 0.8452
Train on 105000 samples, validate on 15000 samples
Epoch 1/100
105000/105000 [==============================] - 213s 2ms/step - loss: 0.6728 - acc: 0.7489 - val_loss: 0.5734 - val_acc: 0.7941

Epoch 00001: _f1_monitor improved from -inf to 0.66749, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/bicudnngruseq/bicudnngruseq_12.h5
Epoch 2/100
105000/105000 [==============================] - 212s 2ms/step - loss: 0.5867 - acc: 0.7884 - v

105000/105000 [==============================] - 212s 2ms/step - loss: 0.6125 - acc: 0.7766 - val_loss: 0.6457 - val_acc: 0.7645

Epoch 00004: _f1_monitor improved from 0.64274 to 0.64515, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/bicudnngruseq/bicudnngruseq_14.h5
Epoch 5/100
105000/105000 [==============================] - 212s 2ms/step - loss: 0.5994 - acc: 0.7831 - val_loss: 0.6551 - val_acc: 0.7653

Epoch 00005: _f1_monitor improved from 0.64515 to 0.64922, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/bicudnngruseq/bicudnngruseq_14.h5
Epoch 6/100
105000/105000 [==============================] - 212s 2ms/step - loss: 0.5697 - acc: 0.7929 - val_loss: 0.6662 - val_acc: 0.7605

Epoch 00006: _f1_monitor did not improve from 0.64922
Epoch 7/100
105000/105000 [==============================] - 212s 2ms/step - loss: 0.6398 - acc: 0.7829 - val_loss: 0.6810 - val_acc: 0.7579

Epoch 00007: _f1_monitor d

105000/105000 [==============================] - 212s 2ms/step - loss: 0.2588 - acc: 0.9202 - val_loss: 0.3481 - val_acc: 0.8933

Epoch 00006: _f1_monitor did not improve from 0.68281
Epoch 7/100
105000/105000 [==============================] - 213s 2ms/step - loss: 0.2458 - acc: 0.9250 - val_loss: 0.3520 - val_acc: 0.8937

Epoch 00007: _f1_monitor improved from 0.68281 to 0.69111, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/bicudnngruseq/bicudnngruseq_17.h5
Epoch 8/100
105000/105000 [==============================] - 212s 2ms/step - loss: 0.2361 - acc: 0.9280 - val_loss: 0.3657 - val_acc: 0.8929

Epoch 00008: _f1_monitor did not improve from 0.69111

Epoch 00008: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
The 17th grain: f1_score - 0.6911091593039062, acc - 0.8937333333333334
Train on 105000 samples, validate on 15000 samples
Epoch 1/100
105000/105000 [==============================] - 213s 2ms/step - loss: 0.5901 -

In [7]:
MODEL_PRE = os.path.join(SAVED_PATH, MODEL_PRE)
# comps_1 = [(LinXiSingleModel, {'model_file': MODEL_PRE + str(i) + '.h5'}) for i in range(len(y_cols))]
# comp_model_1 = LinXiCompositeModel(comps_1)

In [8]:
weights_files = [MODEL_PRE + str(i) + '.h5' for i in range(len(y_cols))]
comp_model.load_weights(weights_files)

In [9]:
val_preds = comp_model.predict(val_x)
test_preds = comp_model.predict(test_x)

In [10]:
f1s = 0
for i, (vy, vp) in enumerate(zip(val_y.values.T + 2, val_preds)):
    f1 = f1_score(vy, vp, average='macro')
    print("The %sth f1: %s" % (i, f1))
    f1s += f1
    
print("The average f1 of val is %s" % (f1s / len(y_cols)))

The 0th f1: 0.6257614793899609
The 1th f1: 0.4912807146720652
The 2th f1: 0.6765336033670779
The 3th f1: 0.6157267989438395
The 4th f1: 0.7506202455980614
The 5th f1: 0.7288753830611063
The 6th f1: 0.6839611464377703
The 7th f1: 0.7291350607436308
The 8th f1: 0.6532072552649792
The 9th f1: 0.6360072713805507
The 10th f1: 0.6427764309683158
The 11th f1: 0.679419608307052
The 12th f1: 0.6922347034889987
The 13th f1: 0.676108752291918
The 14th f1: 0.649223367496238
The 15th f1: 0.6777350780258649
The 16th f1: 0.5067143603607853
The 17th f1: 0.6911091593039062
The 18th f1: 0.5574410442937057
The 19th f1: 0.680701567340874
The average f1 of val is 0.652228651536835


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
res = test.copy()
for index, col in enumerate(LABELS):
    res[col] = test_preds[index] - 2

res.to_csv(RESULT_CSV, index = False)